In [2]:
import robosuite 
import dexmimicgen  

import h5py 
import imageio 
import numpy as np
import time 
import os
import json

'''

1. Loading dual robot environment
'''

def get_env_metadata_from_dataset(dataset_path, ds_format="robomimic"):
    """
    Retrieves env metadata from dataset.

    Args:
        dataset_path (str): path to dataset

    Returns:
        env_meta (dict): environment metadata. Contains 3 keys:

            :`'env_name'`: name of environment
            :`'type'`: type of environment, should be a value in EB.EnvType
            :`'env_kwargs'`: dictionary of keyword arguments to pass to environment constructor
    """
    dataset_path = os.path.expanduser(dataset_path)
    f = h5py.File(dataset_path, "r")
    if ds_format == "robomimic":
        env_meta = json.loads(f["data"].attrs["env_args"])
    else:
        raise ValueError
    f.close()
    return env_meta

dataset_path = "/home/vishal/Volume_E/Active/Gap_year/grand-project-2025/deepak/dexmimicgen/datasets/two_arm_box_cleanup.hdf5"

env_meta = get_env_metadata_from_dataset(dataset_path)
print(env_meta)

write_to_video = False

env_kwargs = env_meta['env_kwargs']
env_kwargs["env_name"] = env_meta["env_name"]
env_kwargs["has_renderer"] = True
env_kwargs["renderer"] = "mjviewer"
env_kwargs["has_offscreen_renderer"] = write_to_video # False # write_video
env_kwargs["use_camera_obs"] = False

env_kwargs.pop("env_lang")

env = robosuite.make(**env_kwargs)

{'env_name': 'TwoArmBoxCleanup', 'env_version': '1.5.1', 'type': 1, 'env_kwargs': {'robots': ['PandaDexRH', 'PandaDexLH'], 'controller_configs': {'type': 'BASIC', 'body_parts': {'right': {'type': 'OSC_POSE', 'input_max': 1, 'input_min': -1, 'output_max': [0.05, 0.05, 0.05, 0.5, 0.5, 0.5], 'output_min': [-0.05, -0.05, -0.05, -0.5, -0.5, -0.5], 'kp': 150, 'damping_ratio': 1, 'impedance_mode': 'fixed', 'kp_limits': [0, 300], 'damping_ratio_limits': [0, 10], 'position_limits': None, 'orientation_limits': None, 'uncouple_pos_ori': True, 'input_type': 'delta', 'input_ref_frame': 'base', 'interpolation': None, 'ramp_ratio': 0.2, 'gripper': {'type': 'GRIP', 'use_action_scaling': False}}}}, 'translucent_robot': False, 'env_configuration': 'parallel', 'reward_shaping': False, 'camera_names': ['agentview', 'robot0_eye_in_hand', 'robot1_eye_in_hand'], 'camera_heights': 84, 'camera_widths': 84, 'has_renderer': False, 'has_offscreen_renderer': True, 'ignore_done': True, 'use_object_obs': True, 'use_

In [1]:
import pose_utils_vishal_from_mimicgen as PoseUtils
import robosuite.utils.transform_utils as T

[robosuite WARNING] No private macro file found! (macros.py:57)
[robosuite WARNING] It is recommended to use a private macro file (macros.py:58)
[robosuite WARNING] To setup, run: python /home/vishal/Volume_E/Active/Gap_year/grand-project-2025/deepak/robosuite/robosuite/scripts/setup_macros.py (macros.py:59)
[robosuite WARNING] Could not import robosuite_models. Some robots may not be available. If you want to use these robots, please install robosuite_models from source (https://github.com/ARISE-Initiative/robosuite_models) or through pip install. (__init__.py:30)
[robosuite WARNING] Could not load the mink-based whole-body IK. Make sure you install related import properly, otherwise you will not be able to use the default IK controller setting for GR1 robot. (__init__.py:40)


In [3]:
'''
Extracting a demonstration from the dataset
'''

data = h5py.File(dataset_path, "r")
print(f"Keys in the dataset: {data['data'].keys()}")
print(f"Number of demonstrations in the dataset: {len(data['data'].keys())}")
demo_id = 'demo_15'
print(f"Length of demo {demo_id}: {data['data'][demo_id]['obs']['robot0_eef_pos'].shape[0]}")
print(f"Observations shape: {data['data'][demo_id]['obs'].keys()}")
print(f"Actions shape: {data['data'][demo_id]['actions'].shape}")
print(f"Actions: {data['data'][demo_id]['actions']}")
print(f"Actions shape: {data['data'][demo_id]['actions'].shape}")
print(f"Action at step 0: {data['data'][demo_id]['actions'][0]}")

action_trajectory = data['data'][demo_id]['actions']
obs_trajectory = data['data'][demo_id]['obs']
states_trajectory = data['data'][demo_id]['states']

eef_poses = data['data'][demo_id]['datagen_info']['eef_pose']
gripper_actions = data['data'][demo_id]['datagen_info']['gripper_action']
target_eef_poses = data['data'][demo_id]['datagen_info']['target_pose']

Keys in the dataset: <KeysViewHDF5 ['demo_0', 'demo_1', 'demo_10', 'demo_100', 'demo_1000', 'demo_1001', 'demo_1002', 'demo_1003', 'demo_1004', 'demo_1005', 'demo_1006', 'demo_1007', 'demo_1008', 'demo_1009', 'demo_101', 'demo_1010', 'demo_1011', 'demo_1012', 'demo_1013', 'demo_1014', 'demo_1015', 'demo_102', 'demo_103', 'demo_104', 'demo_105', 'demo_106', 'demo_107', 'demo_108', 'demo_109', 'demo_11', 'demo_110', 'demo_111', 'demo_112', 'demo_113', 'demo_114', 'demo_115', 'demo_116', 'demo_117', 'demo_118', 'demo_119', 'demo_12', 'demo_120', 'demo_121', 'demo_122', 'demo_123', 'demo_124', 'demo_125', 'demo_126', 'demo_127', 'demo_128', 'demo_129', 'demo_13', 'demo_130', 'demo_131', 'demo_132', 'demo_133', 'demo_134', 'demo_135', 'demo_136', 'demo_137', 'demo_138', 'demo_139', 'demo_14', 'demo_140', 'demo_141', 'demo_142', 'demo_143', 'demo_144', 'demo_145', 'demo_146', 'demo_147', 'demo_148', 'demo_149', 'demo_15', 'demo_150', 'demo_151', 'demo_152', 'demo_153', 'demo_154', 'demo_155'

In [12]:
def just_reset(env, state):
    # The following resets the state
    env.reset()
    # env.sim.set_state_from_flattened(state)
    # env.sim.forward()

    # This part is just to visualize the state after resetting
    env.render()
    obfull = env.step(np.zeros(env.action_dim))
    env.render()
    return obfull

def reset_to_state(env, state):
    # The following resets the state
    env.reset()
    env.sim.set_state_from_flattened(state)
    env.sim.forward()

    # This part is just to visualize the state after resetting
    env.render()
    obfull = env.step(np.zeros(env.action_dim))
    env.render()
    return obfull

In [7]:
'''
Replay a demonstration
'''
reset_to_state(env, states_trajectory[0])
# just_reset(env, states_trajectory[25])

for i in range(action_trajectory.shape[0]):
    action = action_trajectory[i]
    obs, reward, done, info = env.step(action)
    env.render()
    time.sleep(0.05)

In [55]:
'''
Generating novel demonstration
'''
# reset_to_state(env, states_trajectory[0])
obfull = just_reset(env, states_trajectory[25])

max_dpos = 0.05
max_drot = 0.5



In [53]:
obfull[0].keys()
obfull[0]['robot0_eef_pos'], obfull[0]['robot0_eef_quat'], obfull[0]['robot1_eef_pos'], obfull[0]['robot1_eef_quat']

(array([-0.10890595, -0.24315468,  1.11708306]),
 array([ 0.99715776, -0.00586935,  0.07499281, -0.00424655]),
 array([-0.11661582,  0.26369951,  1.11778467]),
 array([ 0.99736291,  0.00731811,  0.07216278, -0.00249229]))

In [54]:
obs = obfull[0]

for i in range(action_trajectory.shape[0]):
    action = action_trajectory[i]

    p0 = PoseUtils.unmake_pose(target_eef_poses[i])
    action[0:3] = p0[0][0] - obs['robot0_eef_pos']
    action[0:3] = np.clip(action[0:3] / max_dpos, -1., 1.)
    # curr_rot = T.quat2mat(obs['robot0_eef_quat'])
    # target_rot = p0[1][0]
    # delta_rot_mat = target_rot.dot(curr_rot.T)
    # delta_quat = T.mat2quat(delta_rot_mat)
    # delta_rotation = T.quat2axisangle(delta_quat)
    # delta_rotation = np.clip(delta_rotation / max_drot, -1., 1.)
    # action[3:6] = delta_rotation

    action[12:15] = p0[0][1] - obs['robot1_eef_pos']
    action[12:15] = np.clip(action[12:15] / max_dpos, -1., 1.)
    # curr_rot = T.quat2mat(obs['robot1_eef_quat'])
    # target_rot = p0[1][1]
    # delta_rot_mat = target_rot.dot(curr_rot.T)
    # delta_quat = T.mat2quat(delta_rot_mat)
    # delta_rotation = T.quat2axisangle(delta_quat)
    # delta_rotation = np.clip(delta_rotation / max_drot, -1., 1.)
    # action[15:18] = delta_rotation

    obs, reward, done, info = env.step(action)
    # obfull[0] = obs
    env.render()
    time.sleep(0.05)


In [37]:
obs['robot0_eef_pos']

array([-0.27634358,  0.43484571,  0.98203987])

In [21]:
p0[1][0]

array([[-0.99597645, -0.08945754, -0.00531572],
       [-0.00507369, -0.00293271,  0.99998283],
       [-0.08947159,  0.99598632,  0.00246703]])